In [4]:
import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import logging
import os
import pandas as pd

import nngp
import neural_net

import bokeh.io
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, gridplot
from bokeh.palettes import Category10
from bokeh.models import Legend

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Experimental Functions

In [5]:
def prep_data(X, Y, dtype=tf.float64):
    X_flat = tf.convert_to_tensor(X.reshape(-1, 28*28)/255, dtype=dtype)
    Y_cat = keras.utils.to_categorical(Y)
    Y_cat = Y_cat - 0.1
    Y_cat = tf.convert_to_tensor(Y_cat, dtype=dtype)
    return X_flat, Y_cat

def truncate_data(n, X, Y):
    return X[:n], Y[:n]

In [6]:
def evaluate_NN(X_train, Y_train, X_test, Y_test, model, training_accuracy=1, max_epochs=400):
    
    callbacks = AccuracyCallback(training_accuracy, max_epochs)
    model.fit(X_train, Y_train, verbose=0, callbacks=[callbacks], epochs=max_epochs)
    output = model.predict(X_test, verbose=0)
    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    _, training_acc = model.evaluate(X_train, Y_train, verbose=0)
    
    return acc, loss, training_acc

In [7]:
class AccuracyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, accuracy_threshold, max_epoch):
        self.accuracy_threshold = accuracy_threshold
        self.max_epoch = max_epoch
        super(AccuracyCallback, self).__init__()
    
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('categorical_accuracy') >= self.accuracy_threshold):   
            print("Reached %2.1f%% accuracy, so stopping training \n" %(self.accuracy_threshold*100))   
            self.model.stop_training = True
        if epoch == self.max_epoch:
            print(f"Stopping training after {epoch} epochs, final accuracy: {logs.get('categorical_accuracy')}")

In [44]:
def evaluate_GP(X_train, Y_train, X_test, Y_test, K):
    mu_bar, _ = nngp.GP_cholesky(X_train, Y_train, X_test, K)

    # Calculate Accuracy
    predictions = tf.argmax(mu_bar, axis=1).numpy()
    predictions_true = tf.argmax(Y_test, axis=1).numpy()
    acc = np.equal(predictions, predictions_true).sum()/len(Y_test)
    
    # MSE Loss
    loss = tf.keras.losses.MSE(mu_bar, Y_test)
    average_loss = loss.numpy().mean()
    return acc, average_loss

In [54]:
def save_bokeh(plot, filepath):
    bokeh.io.reset_output()
    bokeh.io.output_file(filepath, plot)
    show(p)
    bokeh.io.reset_output()
    bokeh.io.output_notebook()

## Experiments

### Comparison of Analytic and General Kernel

In [9]:
def make_norms(vectors):
    """ Makes square of the norm of a vector equal it's dimension
    """
    din = len(vectors[0])
    return np.sqrt(din)*vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

def rotate(vector, theta):
    """ Rotate a 2d vector by an angle theta
    """
    R = np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    return np.matmul(R, vector.T)

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [10]:
def run_angular(vectors, rotated_vectors, K):
    """ Obtain result of applying kernel K to vectors and rotated_vetors in turn
    """

    K_thetas = np.empty((vectors.shape[0],))
    for i, vector_pair in enumerate(zip(vectors, rotated_vectors)):
        K_thetas[i] = K(tf.convert_to_tensor([vector_pair[0]], dtype=tf.float64), 
                        tf.convert_to_tensor([vector_pair[1]], dtype=tf.float64)).numpy()

    return np.squeeze(K_thetas)

In [11]:
n_vectors = 50

In [12]:
# Generate n_vector pairs at a range of angles
angles = np.linspace(0, 2*np.pi, n_vectors)
vectors = np.random.random(size=(n_vectors, 2))
rotated_vectors = np.empty(vectors.shape)
for i, vector in enumerate(vectors):
    rotated_vectors[i,:] = rotate(vector, angles[i])
    
vectors = make_norms(vectors)
rotated_vectors = make_norms(rotated_vectors)

In [13]:
depths = range(0, 9)
sigma_b = np.sqrt(0.1)
sigma_w = np.sqrt(1.6)

# Intialise Kernels
analytical_kernel = nngp.AnalyticalKernel(L=0, sigma_b=sigma_b, sigma_w=sigma_w)
general_kernel = nngp.GeneralKernel(tf.nn.relu, sigma_b=sigma_b, sigma_w=sigma_w ,L=0, n_g=401, n_v=400, n_c=400, u_max=10, s_max=100)

K_thetas_analytical = np.empty((len(vectors), len(depths)))
K_thetas_general = np.empty((len(vectors), len(depths)))

for depth in depths:
    analytical_kernel.L = depth
    general_kernel.L = depth
    K_thetas_analytical[:, depth] = run_angular(vectors, rotated_vectors, analytical_kernel.K)
    K_thetas_general[:, depth] = run_angular(vectors, rotated_vectors, general_kernel.K)

2020-04-25 23:21:51,488 - Loaded grid from file


In [49]:
def plot_angular():
    p = figure(title="Kernel Comparison", x_axis_label='Angle / 2pi', y_axis_label='K(Angle)', plot_width=800, plot_height=450)

    for depth in depths:
        p.line(angles/(2*np.pi), K_thetas_analytical[:,depth], legend_label="Analytical kernel", line_width=2, line_color='blue')
        p.scatter(angles/(2*np.pi), K_thetas_general[:,depth], legend_label="Interpolation", fill_color='red', line_color='red', marker='cross', size=9, line_width=1)
    p.legend.location = "bottom_left"

    show(p)
    return p

In [51]:
p = plot_angular()

In [55]:
save_bokeh(p, 'charts/angular.html')

Loading BokehJS ...

In [15]:
mean = np.mean(vectors, axis=1, keepdims=True)
var = np.var(vectors, axis=1, keepdims=True)
x = (vectors - mean) / tf.sqrt(var)

### Neural Networks vs NNGP on MNIST

In [18]:
def df_append(df, data):
    df.loc[len(df)] = data

In [19]:
def run_over_dataset_and_widths(X_train_flat, Y_train_reg, X_test_flat, Y_test_reg, activations):
    
    

    widths = [5, 10, 50, 100, 500, 1000]
    dataset_size = [20, 100, 250, 500, 1000]
    n_layers = 3

    df_columns = ['activation', 'dataset_size', 'width', 'test_accuracy', 'test_mse', 'training_accuracy']
    nn_evaluate = pd.DataFrame(columns=df_columns)
    gp_evaluate = pd.DataFrame(columns=df_columns)

    # Test size doesn't change in the loop
    n_test = 1000
    X_test_n, Y_test_n = truncate_data(n_test, X_test_flat, Y_test_reg)
    
    for act in activations:
        
        # Kernel settings don't change
        sigma_b = np.sqrt(0.1)
        sigma_w = np.sqrt(1.6)
        general_kernel = nngp.GeneralKernel(act,
                                            L=n_layers,
                                            n_g=401,
                                            n_v=400,
                                            n_c=400,
                                            u_max=10,
                                            s_max=100,
                                            sigma_b=sigma_b,
                                            sigma_w=sigma_w)

        for n, n_data in enumerate(dataset_size):
            print(f'Training Data set size of {n_data} with {act.__name__} activation')

            # Construct the dataset size
            X_train_n, Y_train_n = truncate_data(n_data, X_train_flat, Y_train_reg)
            # Sizes for building nerual net
            din=len(X_train_n[0])
            dout = len(Y_train_n[0])

            # Evaluate the GP first
            gp_acc, gp_loss = evaluate_GP(X_train_n, Y_train_n, X_test_n, Y_test_n, general_kernel.K)
            df_append(gp_evaluate, [act.__name__, n_data, np.nan, gp_acc, gp_loss, np.nan])

            # Evaluate the nn over a range of widths
            for i, width in enumerate(widths):
                print(f'Training model of width: {width} and depth: {n_layers}')
                model = neural_net.build_model((din,), dout, n_layers, width, hidden_activation=act, final_activation=None)
                model.compile(optimizer='Adam', loss='MSE', metrics=['categorical_accuracy'])
                nn_acc, nn_loss, nn_train_acc = evaluate_NN(X_train_n, Y_train_n, X_test_n, Y_test_n, model)
                df_append(nn_evaluate, [act.__name__, n_data, width, nn_acc, nn_loss, nn_train_acc])

    return gp_evaluate, nn_evaluate

In [20]:
# Load the mnist dataset
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

In [21]:
X_train_flat, Y_train_reg = prep_data(X_train, Y_train)
X_test_flat, Y_test_reg = prep_data(X_test, Y_test)

In [22]:
activations = [tf.nn.tanh, tf.nn.relu]
gp_evaluate, nn_evaluate = run_over_dataset_and_widths(X_train_flat, Y_train_reg, X_test_flat, Y_test_reg, activations)

In [27]:
gp_evaluate.to_csv('experimental_data/gp_results.csv')
nn_evaluate.to_csv('experimental_data/nn_results.csv')

In [57]:
def plot_metric(metric, y_label, legend_loc='bottom_right', save_to_file=None):
    widths = nn_evaluate.width.unique()
    activations = ['relu', 'tanh']


    # create three plots
    x_label = 'Training Dataset Size'
    y_label = y_label
    legend=[]
    s1 = figure(title='MNIST, Relu', background_fill_color="#fafafa", x_axis_label = x_label, y_axis_label = y_label)
    s2 = figure(title='MNIST, Tanh', background_fill_color="#fafafa", x_axis_label = x_label)
    s3 = figure()

    color = Category10[len(widths)]

    gp_relu = gp_evaluate[gp_evaluate.activation=='relu']
    gp_tanh = gp_evaluate[gp_evaluate.activation=='tanh']
    s1.line(gp_relu.dataset_size, gp_relu[metric], line_dash=[3], line_width=2)
    r = s2.line(gp_relu.dataset_size, gp_relu[metric], line_dash=[3], line_width=2, legend_label='Gaussian Process')
    # legend.append(('Gaussian Process', [r]))

    for i, width in enumerate(widths):
        nn_relu = nn_evaluate[(nn_evaluate.width==width) & (nn_evaluate.activation=='relu')]
        nn_tanh = nn_evaluate[(nn_evaluate.width==width) & (nn_evaluate.activation=='tanh')]
        s1.line(nn_relu.dataset_size, nn_relu[metric], color=color[i], line_width=2)
        r = s2.line(nn_tanh.dataset_size, nn_tanh[metric], color=color[i], line_width=2, legend_label=f'NN width: {width}')
        legend.append((f'NN width: {width}', [r]))


    # make a grid
    grid = gridplot([s1, s2], ncols=2, plot_width=400, plot_height=350)

    # s2.legend = Legend(items=legend)
    s2.legend.location = legend_loc
    s2.legend.spacing = 0
    # s2.add_layout(legend, 'right')
    s2.legend.orientation = "vertical"
    s2.legend.label_text_font_size = "9px"
    s2.y_range.start = 0
    s1.y_range.start = 0
    
    
    show(grid)
    return grid

In [58]:
p = plot_metric('test_accuracy', 'Test Accuracy')

In [59]:
save_bokeh(p, 'charts/mnist-acc.html')

2020-04-25 23:57:11,726 - Session output file 'charts/mnist-acc.html' already exists, will be overwritten.


Loading BokehJS ...

In [60]:
p = plot_metric('test_mse', 'Mean Squared Error', legend_loc='top_right')

In [61]:
save_bokeh(p, 'charts/mnist-loss.html')

Loading BokehJS ...